# TP1 - Parte B: Classificação de Atividades Humanas 
### Tópicos de Ciência dos Dados - MECD (2022/2023)

### Autores

- Duarte Meneses - 2019216949
- Patricia Costa - 2019213995


In [3]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
from sklearn import datasets
from sklearn.decomposition import PCA
import pandas as pd

In [219]:
columns_labels = [ 'Device ID', 'accelerometer x', 'accelerometer y', 'accelerometer z',
    'gyroscope x', 'gyroscope y', 'gyroscope z', 'magnetometer x', 'magnetometer y', 'magnetometer z',
    'Timestamp', 'Activity Label']

activities = np.array(['Stand', 'Sit', 'Sit and Talk', 'Walk', 'Walk and Talk', 'Climb Stair (up/down)',
    'Climb Stair (up/down) and talk', 'Stand -> Sit', 'Sit -> Stand', 'Stand -> Sit and talk', 'Sit -> Stand and talk',
    'Stand -> walk', 'Walk -> stand', 'Stand -> climb stairs (up/down)(and talk)', 'Climb stairs (up/down) -> walk',
    'Climb stairs (up/down) and talk -> walk and talk'])

sensors = ['Acc', 'Gyr', 'Mag']

## 4.5 e 4.6 da Parte A

## Parte B


### Dataset

In [24]:
# import some data to play with
iris = datasets.load_iris()

data = iris.data[:, :2]  # we only take the first two features.
target = iris.target

### 1.1. Data Splitting

### 1.1.1. Train-Test (TT) e Train-Validation-Test data split

In [25]:
from sklearn.model_selection import train_test_split

def tt(datav, targetv):
    data_train, data_test, target_train, target_test = train_test_split(datav, targetv, test_size=(len(datav)//3), train_size=((2*len(datav))//3), shuffle=True)
    
    return data_train, data_test, target_train, target_test

def tvt(datav, targetv):
    data_train, data_test1, target_train, target_test1 = train_test_split(datav, targetv, test_size=(int(0.6 * len(datav))), train_size=(int(0.4 * len(datav))), shuffle=True)

    data_validate, data_test, target_validate, target_test = train_test_split(data_test1, target_test1, test_size=(len(data_test1)//2), train_size=(len(target_test1)//2), shuffle=True)
    
    return data_train, data_validate, data_test, target_train, target_validate, target_test

data_trainTT, data_testTT, target_trainTT, target_testTT = tt(data, target)
data_trainTVT, data_validateTVT, data_testTVT, target_trainTVT, target_validateTVT, target_testTVT = tvt(data, target)

### 1.1.2. K-fold data split

In [31]:
from sklearn.model_selection import KFold
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold(x, y, n_splits, shuffle = False, random_state = None):
    kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=shuffle)
    #kfold = []
    for train_index, test_index in kf.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        print("TRAIN:", [x_train, y_train])
        print("TEST:", [x_test, y_test])
        print("-" * 100)
        #kfold.append({"TRAIN": [x_train, y_train], "TEST": [x_test, y_test]})
    
    #return kfold
'''
out_kfold = k_fold(data, target, 2)

display(out_kfold)
'''
k_fold(data, target, 2)

TRAIN: [array([[6.6, 3. ],
       [6.8, 2.8],
       [6.7, 3. ],
       [6. , 2.9],
       [5.7, 2.6],
       [5.5, 2.4],
       [5.5, 2.4],
       [5.8, 2.7],
       [6. , 2.7],
       [5.4, 3. ],
       [6. , 3.4],
       [6.7, 3.1],
       [6.3, 2.3],
       [5.6, 3. ],
       [5.5, 2.5],
       [5.5, 2.6],
       [6.1, 3. ],
       [5.8, 2.6],
       [5. , 2.3],
       [5.6, 2.7],
       [5.7, 3. ],
       [5.7, 2.9],
       [6.2, 2.9],
       [5.1, 2.5],
       [5.7, 2.8],
       [6.3, 3.3],
       [5.8, 2.7],
       [7.1, 3. ],
       [6.3, 2.9],
       [6.5, 3. ],
       [7.6, 3. ],
       [4.9, 2.5],
       [7.3, 2.9],
       [6.7, 2.5],
       [7.2, 3.6],
       [6.5, 3.2],
       [6.4, 2.7],
       [6.8, 3. ],
       [5.7, 2.5],
       [5.8, 2.8],
       [6.4, 3.2],
       [6.5, 3. ],
       [7.7, 3.8],
       [7.7, 2.6],
       [6. , 2.2],
       [6.9, 3.2],
       [5.6, 2.8],
       [7.7, 2.8],
       [6.3, 2.7],
       [6.7, 3.3],
       [7.2, 3.2],
       [6.2, 2.8],
    

### 1.2. Métricas de exactidação

### 1.2.1. Matriz de confusão